In [39]:
import time
import requests
from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd

from tqdm import tqdm

In [30]:
###### INIT ######
init_country = 'ru' # ua,au,at,az,al,ar,aw,be,by,bg,ru
init_category = 'films' # films
init_genre = '' # comedy,action,detective,melodrama,thriller,horror,adventures,sport,fiction,crime
init_year = '2021,2020,2016-2019,2010-2015' # 2021,2020,2016-2019,2010-2015,2000-2009,1990-1999,1980-1989,1970-1979,-1969

url_sampler = lambda country = init_country, category = init_category, genre = init_genre, year = init_year: \
    f"""https://megogo.net/ua/search-extended?category_id={category}&country_id={country}&main_tab=filters&sort=popular&video_genre_id={genre}&year_id={year}"""

target_url = url_sampler()

print('default - ', url_sampler())
print('custom - ', url_sampler('en', 'adventure', '2017'))

default -  https://megogo.net/ua/search-extended?category_id=films&country_id=ru&main_tab=filters&sort=popular&video_genre_id=&year_id=2021,2020,2016-2019,2010-2015
custom -  https://megogo.net/ua/search-extended?category_id=adventure&country_id=en&main_tab=filters&sort=popular&video_genre_id=2017&year_id=2021,2020,2016-2019,2010-2015


In [5]:
# target_url = 'https://megogo.ua/ua/search-extended?category_id=series&country_id=ru&main_tab=filters&sort=popular&year_id=2021,2020,2016-2019,2010-2015'

In [31]:
def try_search(driver, target):
    """Trying to find LOAD MORE button. If not exis, return False. List of items is full

    Args:
        driver (_type_): _description_
        target (_type_): _description_

    Returns:
        _type_: _description_
    """
    try:
        driver.find_element_by_class_name(target).click()
        return True
    except:
        return False

In [32]:
##### For remote selenium server only #####

##### To make this work, run :

# wget https://github.com/SeleniumHQ/selenium/releases/download/selenium-4.1.0/selenium-server-4.1.2.jar

# wget https://chromedriver.storage.googleapis.com/98.0.4758.80/chromedriver_linux64.zip

# unzip chromedriver_linux64.zip

# sudo mv -f chromedriver /usr/bin/chromedriver

# wget https://dl.google.com/linux/direct/google-chrome-stable_current_x86_64.rpm

# yum localinstall google-chrome-stable_current_x86_64.rpm

# java -jar selenium-server-4.1.2.jar standalone --allow-cors true - Start selenium remote server 

options = webdriver.ChromeOptions()
options.add_argument("no-sandbox")
options.add_argument("--headless")
options.add_argument("--disable-gpu")

driver = webdriver.Remote(
    command_executor='http://172.30.110.112:4444',
    options=options
)
driver.get(target_url)
target_div = 'pagination-more'

while try_search(driver, target_div):
    time.sleep(1)

items = driver.find_elements_by_class_name('videoItem')
html_items = [item.get_attribute("outerHTML") for item in items]

driver.quit()

/tmp/ipykernel_7417/3284241410.py:12: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  driver.find_element_by_class_name(target).click()
/tmp/ipykernel_7417/4196354492.py:34: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  items = driver.find_elements_by_class_name('videoItem')


In [34]:
# len(html_items)

1270

In [35]:
obj = BeautifulSoup(html_items[0], 'lxml')
obj_ = BeautifulSoup(requests.get(list(obj.a.attrs.values())[0]).text, 'lxml')
obj_.findAll('div', {'class' : 'video-description'})[0].text

"\n    Заради сім'ї Гена залишив позаду свою молодість та мрію – стати успішним гравцем КВК. Через двадцять п'ять років дружина вирішила, що їхнє життя нудне, і запропонувала подати на розлучення. Чоловік збирається надолужити згаяний час і вирушає разом із новою знайомою на фестиваль у Сочі. Щоправда, дівчина планує в будь-який момент зникнути разом із його грошима...\n    "

In [ ]:

# ! Названия, год, страна, айтем url , / жанр 

In [37]:
def parse_megogo(html_item, parse_country: True):
    """Parsing html data from Megogo site. If parse_country passed True 
    and items list are big, site may block requests.

    Args:
        html_item (_type_): _description_
        parse_country (True): _description_

    Returns:
        _type_: _description_
    """
    try:
        obj = BeautifulSoup(html_item, 'lxml')
    except Exception (e):
        print(e)
        return [None] * 6 
    
    if parse_country:
        try:
            obj_page = BeautifulSoup(requests.get(list(obj.a.attrs.values())[0]).text, 'lxml')
            country = obj_page.findAll('a', {'class' : 'video-country link-default'})[0].text
            description = obj_page.findAll('div', {'class' : 'video-description'})[0].text
        except:
            country = None
            description = None
    else:
        country = None
        
    try:
        item_url, name = list(obj.a.attrs.values())
    except:
        item_url, name = None, None
        
    try:
        genre, year = obj.findAll('span', {'class' :  'video-country'})[0].text, \
            obj.findAll('span', {'class' :  'video-year'})[0].text
    except:
        genre, year = None, None
    time.sleep(0.1)
    print([item_url, name, year, country, genre])
    return [item_url, name, year, country, genre, description]

In [38]:
res = pd.DataFrame([parse_megogo(item, parse_country = True) for item in tqdm(html_items)], \
                columns = ['item_url', 'name', 'year', 'country', 'genre', 'description'])

['https://megogo.net/ua/view/12300695-chesne-rozluchennya.html', 'Чесне розлучення', '2021', 'Росія', 'Комедії']
['https://megogo.net/ua/view/14765855-svingeri.html', 'Свінгери', '2021', 'Росія', 'Комедії']
['https://megogo.net/ua/view/7733765-smertelni-ilyuzi.html', 'Смертельні ілюзії', '2020', 'Росія', 'Трилери']
['https://megogo.net/ua/view/7979075-tin-zirki.html', 'Тінь зірки', '2020', 'Росія', 'Детективи']
['https://megogo.net/ua/view/244571-ostriv-vezinnya.html', 'Острів везіння', '2013', 'Росія', 'Комедії']
['https://megogo.net/ua/view/4476021-boloviy-porig.html', 'Больовий поріг', '2019', 'Росія', 'Трилери']
['https://megogo.net/ua/view/2994631-trener.html', 'Тренер', '2018', 'Росія', 'Спортивні']
['https://megogo.net/ua/view/14597805-shugaley-3.html', 'Шугалей 3', '2021', 'Росія', 'Бойовики']
['https://megogo.net/ua/view/3497551-iz-dna-vershini.html', 'Із дна вершини', '2018', 'Росія', 'Спортивні']
['https://megogo.net/ua/view/2864831-gulyay-vasya.html', 'Гуляй, Вася!', '2017'

In [41]:
res['type'] = 'film'

In [23]:
res.columns = ['genre', 'item_url', 'name', 'country', 'year', 'description']
res = res[['item_url', 'name', 'year', 'country', 'genre', 'description']]

In [42]:
# res['type'] = 'series'
# megogo_series
megogo_film = res

In [49]:
megogo_film.type.unique()

array(['film'], dtype=object)

In [92]:
import re


any(list(map(int, re.findall('\d{4}', '2017')))) < 2014

True

In [95]:
any(list(map(int, re.findall('\d{4}', '2017'))))  < 2014

True

In [88]:
any(map(int, re.findall('\d{4}', megogo_export.year.values[23]))) < 2014

True

In [89]:
megogo_export.year.values

array(['2021', '2021', '2020', ..., '2009 - 2010', '2011', '2010'],
      dtype=object)

In [90]:
[any(list(map(int, re.findall('\d{4}', i)))) < 2014 for i in megogo_export.year.values[0]] 

[True, True, True, True]

In [77]:
megogo_export = pd.concat([megogo_film,  megogo_series], axis = 0)

megogo_export.year.unique()

# megogo_export.year = megogo_export.year.astype(int)
megogo_export = megogo_export[[any(list(map(int, re.findall('\d{4}', i)))) < 2014 for i in megogo_export.year.values] ]
# megogo_export[megogo_export.year >= 2014]
# .to_csv(path + 'megogo_export.csv')

TypeError: expected string or bytes-like object

In [29]:
path = '/home/vmalyniak/home/jupyter/parse_megogo/'
# res.to_csv(path + f'megogo_export_{time.ctime()}.csv')
# megogo_series.to_csv(path + 'megogo_series.csv')